# 朴素贝叶斯

## 作用

​	对于给定的数据集，首先基于特征条件假设学习输入输出的联合概率分布，然后基于这个分来算出给定的输入$x$,利用贝叶斯定理求出后验概率最大的输出$y$。





## 方法

​	输入空间$\mathcal{X}\subset{R^n}$,是一个$n$维向量的集合，输出空间$\mathcal{Y}=\{c_1,c_2...C_K\}$是一个类标记的集合。$X$定义为在输入空间的一个随机向量，$Y$定义为输出空间的一个随机值，计算$X,Y$的联合概率分布$P(X,Y)$。





### 联合概率分布的计算

​		假设训练数据集$T=\{(x_1,y_1),(x_2,y_2)...(x_N,y_N)\}$是由$P(X,Y)$独立同分布产生。

​	1.计算先验概率分布：
$$
P(Y=c_k)=\frac{\sum_{i=1}^NI(y_i=c_k),符合条件的个数}{N,样本总数},k=1,2,...K
$$
​	2.计算条件概率分布：
$$
P(X=x|Y=c_k)=P(X^{(1)}=x^{(1)},...X^{(n)}=x^{(n)}|Y=c_k)\\
\quad\qquad=\prod_{j=1}^nP(X^{(j)}=x^{(j)}|Y=c_k)\\
\quad\qquad\qquad=\prod_{j=1}^n\frac{x_i^j=a_{jl}并且y_i=c_k的个数}{y_i=c_k的个数}
$$





### 分类任务

​	用朴素贝叶斯做分类时，对于给定的输入$x$，通过学习到的模型来计算后验概率分布$P(Y=c_k|X=x)$，将后验概率最大的类作为输出。
$$
y=f(x)=argmax(P(Y=c_k|X=x)) \\
\\
= argmax(\frac{P(Y=c_k)\prod_{j=1}^nP(X^{(j)}=x^{(j)}|Y=c_k)}

{\sum_kP(Y=c_k)\prod_{j=1}^nP(X^{(j)}=x^{(j)}|Y=c_k)})
$$
​	由于计算的是argmax，所以可以将共有的分母取消，最终公式为：
$$
y=argmax(P(Y=c_k)\prod_{j=1}^nP(X^{(j)}=x^{(j)}|Y=c_k))
$$
​	后验概率最大化等价于期望概率的最小化。



# 贝叶斯库使用

## 库介绍

​	在scikit-learn中，一共有3个朴素贝叶斯的分类算法类。分别是GaussianNB，MultinomialNB和BernoulliNB。其中GaussianNB就是先验为高斯分布的朴素贝叶斯，MultinomialNB就是先验为多项式分布的朴素贝叶斯，而BernoulliNB就是先验为伯努利分布的朴素贝叶斯。





## GaussianNB

​	GaussianNB会根据训练集求出高斯分布的两个参数，$\mu_k$是样本$C_k$中所有$X_j$的平均值，$\sigma_k^2$是所有$X_j$的方差，GaussianNB类的主要参数仅有一个，即先验概率priors ，对应$Y$的各个类别的先验概率$P(Y=C_k)$，如果没有给出先验概率，那么先验概率的计算通过，输出为第k类别的训练集样本数除以从样本数。

​	在使用GaussianNB的fit方法拟合数据后，我们可以进行预测。此时预测有三种方法，包括predict，predict_log_proba和predict_proba。

​	predict方法就是我们最常用的预测方法，直接给出测试集的预测类别输出。

​	predict_proba则不同，它会给出测试集样本在各个类别上预测的概率。容易理解，预测出的各个类别概率里的最大值对应的类别，也就是predict方法得到类别。

​	predict_log_proba和predict_proba类似，它会给出测试集样本在各个类别上预测的概率的一个对数转化。转化后predict_log_proba预测出的各个类别对数概率里的最大值对应的类别，也就是predict方法得到类别。

In [4]:
import numpy as np

X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
Y = np.array([1, 1, 1, 2, 2, 2])
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
#拟合数据
clf.fit(X, Y)
print("==Predict result by predict==")
print(clf.predict([[-0.8, -1]]))
print("==Predict result by predict_proba==")
print(clf.predict_proba([[-0.8, -1]]))
print("==Predict result by predict_log_proba==")
print(clf.predict_log_proba([[-0.8, -1]]))

==Predict result by predict==
[1]
==Predict result by predict_proba==
[[9.99999949e-01 5.05653254e-08]]
==Predict result by predict_log_proba==
[[-5.05653266e-08 -1.67999998e+01]]


## MultinomialNB

​	这种情况下，假设的先验概率分布式子为：
$$
P(X_j=x_{jl}|Y=C_k)=\frac{x_{jl}+\lambda}{m_k+n\lambda}\\
\lambda:是一个大于0的常数，默认为1，即拉普拉斯平滑，
如果拟合效果不好，可以尝试调试稍大或稍小于1的值
$$
​		两个参数fit_prior和class_prior，fit_prior是一个布尔值，来选择是否要指定每个类别的先验概率，如果不指定，则每个类别都有同一个先验概率也就是1/类的个数k，如果指定fit_prior，并且class_prior有值，则先验概率为class_prior,否则先验概率就是第k类别的训练集样本数/总样本数：

| fit_prior | class_prior  | 先验概率                       |
| --------- | ------------ | ------------------------------ |
| False     | 填不填无意义 | 1/类的个数k                    |
| True      | 不填         | 第k类别的训练集样本数/总样本数 |
| True      | 填一个数     | class_prior                    |





## BernoullinNB

​	假设先验概率为二维伯努利分布：
$$
P(X_j=x_{jl}|Y=C_k)=P(j|Y=C_k)x_{jl}+(1-P(j|Y=C_k))(1-x_{jl})，x_{jl=0,1}
$$
​	多出的一个参数为binarize，如果不输入就默认每个数据特征都是二元的。如果输入，则把binarize视作一个阈值，大于或小于这个阈值的输入被归为两类。

